In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from random import choice
# Importing Libraries

import sys
import re
import numpy as np
import tensorflow as tf
import os

from tensorflow.keras import activations 
from tensorflow.keras.layers import MaxPool1D, MaxPool2D,LeakyReLU, Activation, Conv1D, InputLayer, Conv2D, Dense, Dropout, Flatten, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow import keras
import tqdm
import random
from random import choices

# Reading human genome chromosome 1

with open("chr 1",'r') as fp:
    genome = fp.readlines()

len(genome)

# <center >Reading the data 

In [2]:
def read_fasta(fp):
    name, seq = None, []
    for line in fp:
        line = line.rstrip()
        if line.startswith(">"):
            if name: yield (name, ''.join(seq))
            name, seq = line, []
        else:
            seq.append(line)
    if name: yield (name, ''.join(seq))
        
        
def len_check(temp, length):
    count = 0
    for i in range(len(temp)):
        if (len(temp[i]) > length):
            count+=1
            
    print("count =", count)

In [3]:
sequences = list()

with open('permissive_enhancers') as fp:
    for name, seq in read_fasta(fp):
        sequences.append(seq)
print("Sequences Read Succesfully !!!!")

Sequences Read Succesfully !!!!


In [4]:
print("Total Permissive Enhancers: ", len(sequences))

Total Permissive Enhancers:  43011


In [5]:
print("Sequences are in unequal length")
len(sequences[10])

Sequences are in unequal length


153

# <center> Information about the data 

In [6]:
sum = 0.0
max = 0
min = 2860
counter = 0
for i in range(len(sequences)):
    sum = sum + len(sequences[i])
    if (len(sequences[i])>max):
        max = len(sequences[i])
    if (len(sequences[i])<min):
        min = len(sequences[i])
        
    if (len(sequences[i])>300):
        counter+=1
average = sum/len(sequences)

print("sum is:", sum, "\naverage is:", average, "\nmax length is:", max, "\nmin length is:", min)
counter

sum is: 12385403.0 
average is: 287.9589639859571 
max length is: 2860 
min length is: 2


18700

# <center> Sub sampling from the data

#### <center> We want to get sequences of length L, if sequences are bigger than L then we try to produce subsequences of size L from these sequences. On the other hand side we try to augment the shorter sequences by nucleotides present in their start.

In [7]:
shorter_sequences = list()
longer_sequences = list()

In [8]:
for i in range(len(sequences)):
    if (len(sequences[i])>300):
        longer_sequences.append(sequences[i])
    elif (len(sequences[i])<=300):
        shorter_sequences.append(sequences[i])

In [9]:
print(len(shorter_sequences), len(longer_sequences))

24311 18700


In [10]:
24311 +18700

43011

#### <center >creating more sequences from longer sequences 

In [11]:
def long_to_augmentation(longer_sequences):
    for i in range (len(longer_sequences)):
        if (len(longer_sequences[i])>300):
            temp = longer_sequences[i]
            longer_sequences[i]=temp[:300]
            longer_sequences.append(temp[300:])
    return longer_sequences


In [12]:
def check_len_seq_long(longer_sequences):
    for j in tqdm(range(len(longer_sequences))):
        if (len(longer_sequences[j])<300):
            shorter_sequences.append(longer_sequences[j])    
    return count_index


In [13]:
for i in range(15):
    longer_sequences=long_to_augmentation(longer_sequences)


In [14]:
len(longer_sequences[9352])

300

In [16]:
print("sequences have been addded in shorter sequences")
length = len(longer_sequences)

new_longer_sequences=[x for x in longer_sequences if len(x)>=300]
        
print("new shorter sequences have been deleted from longer sequences")
####################
print("total long sequences:", len(new_longer_sequences))


sequences have been addded in shorter sequences
new shorter sequences have been deleted from longer sequences
total long sequences: 20796


In [17]:
len(longer_sequences[9350])

300

In [18]:
len(shorter_sequences)

24311

In [19]:
#junk values
#copy paste 
#check from genome

#### <center > augmenting the sequences with shorter length

#### Augmenting randomly

In [20]:
new_shorter_sequences= list()

def augment_shorter_sequences(shorter_sequences):
    for i in range(len(shorter_sequences)):
        if (len(shorter_sequences[i])<300):
            temp = shorter_sequences[i]
            diff = 300-len(shorter_sequences[i])
            for i in range(diff):
                temp += choice('ATCG')
            new_shorter_sequences.append(temp)

#### Augmenting by copy paste

def augment_shorter_sequences(shorter_sequences):
    for i in range(len(shorter_sequences)):
        if (len(shorter_sequences[i])<300 and len(shorter_sequences[i])>=150):
            temp = shorter_sequences[i]
            diff = 300-len(shorter_sequences[i])
            temp = temp+temp[0:diff]
            shorter_sequences[i]=temp
        elif (len(shorter_sequences[i])<300 and len(shorter_sequences[i])>=150):
            temp = temp = shorter_sequences[i]


        
    return shorter_sequences

#### Augmenting by taking into account the real genome sequence

def augment_shorter_sequences(shorter_sequences):
    for i in range(len(shorter_sequences)):
        if (len(shorter_sequences[i])<300 and len(shorter_sequences[i])>=150):
            temp = shorter_sequences[i]
            diff = 300-len(shorter_sequences[i])
            temp = temp+temp[0:diff]
            shorter_sequences[i]=temp
        elif (len(shorter_sequences[i])<300 and len(shorter_sequences[i])>=150):
            temp = temp = shorter_sequences[i]


        
    return shorter_sequences

for i in range(len(shorter_sequences)):
    if (len(shorter_sequences[i])<300):
        temp = shorter_sequences[i]
        diff = 300-len(shorter_sequences[i])
        for i in range(diff):
            temp += choice('ATCG')
        new_shorter_sequences.append(temp)
        #shorter_sequences[i]=temp
        #print(len(shorter_sequences[i]))

In [136]:
augment_shorter_sequences(shorter_sequences)

In [137]:
len(shorter_sequences)

43001

In [138]:
43001 + 20796

63797

# <center> Combining the short and long sequences

In [21]:
def combine(longer_sequences, shorter_sequences):
    dataset = longer_sequences + shorter_sequences
    return dataset
    

In [22]:
datatset = combine(new_longer_sequences, new_shorter_sequences)

In [23]:
len(datatset)

20796

# <center> Sequences to one hot encoding

In [24]:
def baseToOnehot(base):
    if base == 'A':
        return np.array([1,0,0,0])
    elif base == 'G':
        return np.array([0,1,0,0])
    elif base == 'C':
        return np.array([0,0,1,0])
    elif base == 'T':
        return np.array([0,0,0,1])

In [25]:
def one_hot_encode(seq):
    mapping = dict(zip("AGCT", range(4)))    
    seq2 = [mapping[i] for i in seq]
    return np.eye(4)[seq2]

In [27]:
all_samples = np.zeros((len(datatset), 300, 4))
current_sample = np.zeros((300,4))
for i in range(len(datatset)):       
    all_samples[i]=one_hot_encode(datatset[i])

In [28]:
all_samples = all_samples.astype('float32')
print(all_samples.shape)

(20796, 300, 4)


In [29]:
all_samples = np.expand_dims(all_samples, axis=0)


In [30]:
np.save("one_hot_permissive_enhancers",all_samples)

In [31]:
print(all_samples.shape)

(1, 20796, 300, 4)


In [32]:
temp = np.reshape (all_samples, (4,63671,300))


ValueError: cannot reshape array of size 24955200 into shape (4,63671,300)

In [33]:
temp[:,1].shape

NameError: name 'temp' is not defined

In [184]:
#final_samples = np.reshape (all_samples, (63671,4,1,300))


In [185]:
#final_samples[1]